In [1]:
import pandas as pd
import requests as rqs
import os
import json
import pymongo
import datetime
from datetime import timedelta

myclient = pymongo.MongoClient(f'mongodb://{os.getenv("MONGO_URL","192.168.2.1")}:27017',username=os.getenv("MONGO_USERNAME","hterra"),password=os.getenv("MONGO_PASSWORD","terrac0t@"), unicode_decode_error_handler='ignore')
activitywatch_db = myclient["activitywatch"]
aw_watcher_window_db = myclient["activitywatch"]["aw-watcher-window"]


In [43]:
df = pd.DataFrame(list(aw_watcher_window_db.find()))
df.set_index("_id", inplace=True)
df = df[(df['timestamp'] >= '2020-02-01') & (df['timestamp'] < '2020-03-01') & (df['duration'] > 0) & ('.tmp' not in df['app'])& (df['title'] != "")]
#df['user'] = df.apply(lambda x: activitywatch_db['host'].find_one({"host": x['host']})['user'], axis=1)
df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)
df['app'] = df['app'].str.upper()
df['app'] = df['app'].str.replace(".EXE","")
df = df.groupby(['host','timestamp','app','title'], as_index=False)['duration'].sum()


In [44]:
data = df
data['time'] = data['duration'].apply(lambda x:'{:.0f} hr {:.0f} min {:.0f} secs'.format(divmod(x,60*60)[0],*divmod(divmod(x,60*60)[1],60)))

data


host                        timestamp       app  \
0      DESKTOP-1V9F1OU 2020-02-04 14:52:54.166000+00:00   LOCKAPP   
1      DESKTOP-1V9F1OU 2020-02-04 14:54:32.799000+00:00   LOCKAPP   
2      DESKTOP-1V9F1OU 2020-02-04 14:55:27.376000+00:00  SEARCHUI   
3      DESKTOP-1V9F1OU 2020-02-04 14:56:00.710000+00:00  EXPLORER   
4      DESKTOP-1V9F1OU 2020-02-04 14:56:04.974000+00:00   OUTLOOK   
...                ...                              ...       ...   
89809  DESKTOP-SV6ATEN 2020-02-28 21:56:01.983000+00:00   OUTLOOK   
89810  DESKTOP-SV6ATEN 2020-02-28 21:56:04.143000+00:00  IEXPLORE   
89811  DESKTOP-SV6ATEN 2020-02-28 21:56:34.764000+00:00  IEXPLORE   
89812  DESKTOP-SV6ATEN 2020-02-28 21:56:39.207000+00:00  IEXPLORE   
89813  DESKTOP-SV6ATEN 2020-02-28 21:56:48.011000+00:00  IEXPLORE   

                                                   title  duration  \
0          Pantalla de bloqueo predeterminada de Windows    65.779   
1          Pantalla de bloqueo predeterminada de Windows    11.399   
2                                                Cortana    32.287   
3                                               Imágenes     1.062   
4      Bandeja de entrada - karla.rosado@hilcoterraco...    56.318   
...                                                  ...       ...   
89809  Avisos terracota - lorena.duran@hilcoterracota...     1.104   
89810  Empleos en Los Cabos, BCS - Feb 2020 | Jora - ...    29.548   
89811  Empleos: analista documental en Los Cabos, BCS...     3.359   
89812  Empleos: auditor en Los Cabos, BCS - Feb 2020 ...     7.733   
89813  Empleos: chofer en Los Cabos, BCS - Feb 2020 |...    14.217   

                     time  
0       0 hr 1 min 6 secs  
1      0 hr 0 min 11 secs  
2      0 hr 0 min 32 secs  
3       0 hr 0 min 1 secs  
4      0 hr 0 min 56 secs  
...                   ...  
89809   0 hr 0 min 1 secs  
89810  0 hr 0 min 30 secs  
89811   0 hr 0 min 3 secs  
89812   0 hr 0 min 8 secs  
89813  0 hr 0 min 14 secs  

[89814 rows x 6 columns]